# Timing and profiling python

Writing efficient code is a difficult matter. High-level programming languages (like python) provide multiple ways of performing the same tasks. Certain programming choices can be faster, more reliable and more efficient than others. And assessing the efficiency of different implementations is an important skill to learn.
Beware however not to indulge to much into code optimisation at the early stage of development. A famous quote from Donald Knuth states, "We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil."

That said, let's dig into the most useful python libraries for profiling and timing our code.
Within the Jupyter (or IPython) environment, we will explore the capabilities of the following **magic commands**:
* `%time`: elapsed time of a statement *single call*
* `%timeit`: average elapsed time of a statement *multiple calls* (for better accuracy)
* `%prun`: elapsed time *line-by-line* of a statement
* `%lprun`: similar to the above, but more precise
* `%memit`: memory usage of a single statement
* `%mprun`: memory usage of a single statement, line-by-line

The last three commands are not part of the default python distribution and must be manually installed.
The packages required are `line_profiler` and `memory_profiler`. These can be easily installed with `pip`

`$ pip install line_profiler`

Or with `anaconda`

`$ conda install -c anaconda memory_profiler`

(sorry, don't know how to do that in Windows).

Once installed , they can be imported into Jupyter with the command

`%load_ext line_profiler` and `%load_ext memory_profiler`

Note that calling Jupyter magic commands with one `%` only evaluates one statement (line-magic commands).
For evaluating snippets of code use double `%%` (cell-magic commands).

Finally, by appending `?` to a command, we can get some help on how to use it: for example `%timeit?` displays the usage of the `%timeit` magic command.

# `%timeit`

Let's start with an example. We want to compute the sum of all the values of a list of numbers.
There are several way to do this:
* we can write our own summing function
* we can use the built-in python `sum()` command
* we can import numpy and call the `numpy.sum()` method.

To profile these three implementations we will use the `%timeit` magic command. By simply typing

`%timeit <some python statement>`

we can resolve how much time the python statement takes for execution.

Let's try.

In [ ]:
# import numpy
import numpy as np

# defining our user-defined summing function
def mysum(some_list):
    tot = 0
    for value in some_list:
        tot += value
    return tot

In [ ]:
# our list of numbers, python style
mylist = list(range(100))

print(type(mylist))

%timeit s = mysum(mylist)
%timeit s = sum(mylist)
%timeit s = np.sum(mylist)

In [ ]:
# our list of numbers, numpy style
mylist = np.arange(100)

print(type(mylist))

%timeit s = mysum(mylist)
%timeit s = sum(mylist)
%timeit s = np.sum(mylist)

## What happened?

The output of `timeit` shows the elapsed time for an average calculation plus or minus some error.
This is perfomed automatically, since many loops are taken in order to achive sufficient accuracy.

In the above cells, we profiled `mysum`, `sum` and `numpy.sum` for two different cases:
* a python `list` of numbers
* a `numpy.ndarray` of numbers

of same length.

Our function performs pretty bad in both cases. For example, in my machine, summing up a 100-element array took 6 µs and 37 µs for the `list` and `numpy.ndarray` case, respectively.

On the other hand, the built-in `sum()` performed **extrimely** well with the `list` (1 µs), but terribly with `numpy.ndarray` (30 µs).

Similarly, `numpy.sum()` peformed very well with `numpy.ndarray` (6 µs), but **extrimely** bad (worst than our function) with the `list` (21 µs).

We conclude that user define functions, if not properly optimized, must be avoided when a similar built-in function or method is available. Also, we see that mixing different frameworks (i.e. summing `ndarray` with `sum()` or using `numpy.sum()` with `list`) is also a bad idea.

However, why does `sum()` perform **better** than `numpy.sum()` still?

Let's try to change the length of the list.

We write a simple program that iterates over some array lengths and stores the elapsed time in a list.
Please note that the `%timeit` magic command does not allow to return the elapsed time interactively.
However we can use the python library `time` to access the function [`time.perf_counter()`](https://docs.python.org/3/library/time.html#time.perf_counter).
This function is what `timeit` calls by [default](https://docs.python.org/3/library/timeit.html#timeit.default_timer).

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib notebook

Ns = [100, 200, 500, 1000, 2000, 5000, 10000]

python_times = []
numpy_times = []

for N in Ns:
    mylist = list(range(N))
    tot = 0
    for i in range(1000):
        start = time.perf_counter()          # start
        s = sum(mylist)
        tot += (time.perf_counter() - start) # stop
    python_times.append(tot*1000)
    
    mylist = np.arange(N)
    tot = 0
    for i in range(1000):
        start = time.perf_counter()          # start
        s = np.sum(mylist)
        tot += (time.perf_counter() - start) # stop
    numpy_times.append(tot*1000)

plt.plot(Ns, python_times, label='Python')
plt.plot(Ns, numpy_times, label='Numpy')
plt.xlabel('Length')
plt.ylabel('Time (µs)')
plt.legend()
plt.show()

## Impressive!

Over the long run, `numpy` performs **incredibily better** than standard `python`. This has to do with the initial *overhead* of `numpy` objects and to the fact that `numpy` methods are written in precompiled C language.

The take home message is try to avoid user-defined function and instead use built-in methods. Also, pay attention to the scaling of those methods, since the intrinsic overhead of python objects can consume unnecessary time resources.

<div class="alert alert-success">
<b>TRY IT YOURSELF!</b><br>
</div>

Try to profile some of the most common task used in scientific programmming and find out which is better. Below are some examples:
* array initialisation: `range`, `list`, `numpy.arange()`, `numpy.zeros()`
* concatenation: `list1 + [list2]`, `mylist.append()`, `numpy.append()`, `numpy.concatenate()`
* aggregations: `min`, `max`, `mean`, etc
* sorting `mylist.sort()`, `myarray.sort()`, `numpy.sort()`
* indexing and masks 
* broadcasting

The following are some of my attempts.

In [ ]:
# array initialisation
%timeit arr = range(1000)
%timeit arr = np.arange(0,10,0.01)
%timeit arr = np.linspace(0,10,1000,endpoint=False)
%timeit arr = np.empty(1000)
%timeit arr = np.zeros(1000)

In [ ]:
# lets generate some ndarrays and some python lists

rng = np.random.default_rng(0) # random number generator, we set a seed for reproducibility

arr1 = rng.random(10000)
arr2 = rng.random(10000)

list1 = list(arr1)
list2 = list(arr2)

In [ ]:
# list <-> ndarray conversion
%timeit arr = list(arr2)
%timeit arr = np.array(arr1)

In [ ]:
# max and np.max
%timeit max(arr1)
%timeit np.max(arr1)
%timeit max(list1)
%timeit np.max(list1)

In [ ]:
# appending to list
# NOTE: we perform only one repetition and only one loop, because append() replace the list!!!
%timeit -n 1 -r 1 list1 + [0.9387523] # avoid this!
%timeit -n 1 -r 1 list1.append(0.9387523)
%timeit -n 1 -r 1 list1 + list2       # avoid this!
%timeit -n 1 -r 1 list1.extend(list2)

# appending to ndarrays
%timeit np.append(arr1,0.9387523)
%timeit np.concatenate((arr1,[0.9387523]))
%timeit np.append(arr1,arr2)
%timeit np.concatenate((arr1, arr2))       # append uses concatenate, concatenate is faster

In [ ]:
%timeit rng.random(10000)       # new policy
%timeit np.random.random(10000) # legacy

In [ ]:
# sorting
# NOTE: list.sort() and ndarray.sort() replace the array!!
%timeit -n 1 -r 1 list1.sort()
%timeit -n 1 -r 1 arr1.sort()
%timeit -n 1 -r 1 np.sort(arr2) # returns a copy!

# `%prun` and `%lprun`

The mafic command `%prun` is useful to profile the elapsed time of snippets of code line-by-line.
For example, let's define some function that computed the mean and the variance of a collection of data:

In [ ]:
# average and variance of a collection of data
# data is a 2-d array, each row is an independent set of measurements
def meanvar(data):
    mean = []
    variance = []
    for d in data:
        mean.append(np.mean(d))
        variance.append(np.var(d))
    return np.array(mean), np.array(variance)

In [ ]:
# let's create our sample data
# say, 300 sets of 10000 elements each
data = rng.random(size=(300,10000))

In [ ]:
%prun mean, var = meanvar(data)

As you can see, the `%prun` first return the number of calls and the total elapsed time (0.036 seconds in my machine).
Also, it returns several useful information on the commands in our function, these are:
* `ncalls`: number of single calls of our function methods
* `tottime`: total time elapsed for all the method calls
* `percall`: elapsed time per single call
* `cumtime` and `percall`: same as the previous two, just cumulative.

Results are returned in decreasing `tottime` order. We see, for example, that the most time consuming operation is calculating the variance `np.var()`. On my machine, this took 0.013 seconds (out of the 0.036 total time), which makes it 36% of the entire elapsed time!

## Speed it up

Lets try to get rid of the for loop using the capabilities of `numpy`.

We can specify a specific `axis` over which to perform `mean` and `var`. Our function becomes the following.

In [ ]:
def optimal_meanvar(data):
    return np.mean(data, axis=1), np.var(data, axis=1)

Let's run again `%prun`.

In [ ]:
%prun mean, var = optimal_meanvar(data)

Also, let's `%timeit` the two functions.

In [ ]:
%timeit mean, var = meanvar(data)
%timeit mean, var = optimal_meanvar(data)

## Impressive

We got a perforance improvement of 50%! (on my machine)

Finally, let's have a look at the `%lprun` command. First let's import the magic command with

`%load_ext line_profiler`

and then run

`%lprun -f <function_name> <python_statement>`

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f meanvar mean, var = meanvar(data)

In [ ]:
%lprun -f optimal_meanvar mean, var = optimal_meanvar(data)

`%lprun` acts in a similar way to `%prun`. It displays number of call, total time and time per call. The breakdown of the methods is more neat than the previous one, but the information given is less. I suggest to use both commands to have a clear insight on what python is doing under the hood when you want to optimise you code.

# `%memit` and `%mprun`

Finally let's take a look on memory usage. The magic command `%memit` measures how much memory a single python statement uses, while the `%mprun` magic command performs the same task line-by-line.

First, let's import `memory_profiler` with

`%load_ext memory_profiler`

Usage is similar to the previously discussed commands:

`%memit <python_statement>`

`%mprun -f <function_name> <python_statement>`

Let's start with an example.

In [ ]:
%load_ext memory_profiler

In [ ]:
# how much memory does this take
%memit pollo = np.full(1000000,1-1j,dtype='complex')

`memit` displays two values: peak memory is the memory of this entire Jupyter notebook, incrememt is the memory increment due to our command. In this case we initialized an array of 1000000 complex numbers, each complex number uses 128 bits of memory (by default numpy uses `complex128`), 8 bits per bytes, 1024 bytes per kB, 1024 kB per MB: correct, we get exactly 15.26 MB.

Now let's look at the line-by-line memory profiler.
To use `%mprun`, we need to import the function to profile *from file*. So let's create a file with `%%file`, define a function and import it.

In [ ]:
%%file file_with_my_function.py

import numpy as np

def psi(x):
    y = np.exp(-x**2)
    A = np.trapz(y**2, x)
    y /= np.sqrt(A)
    return y

In [ ]:
import file_with_my_function as fil

In [ ]:
%memit x = np.linspace(-10,10,1000000)

In [ ]:
%mprun -f fil.psi psi = fk.psi(x)

`%mprun` breaks down each command of our user-define function and shows how much memory each statement uses.

Final note, `numpy` functions such as `numpy.exp()`, `numpy.multiply()`, etc, save their return value into a temporary memory slot before returning it to the assigned variable. To avoid this waste of memeory (especially when memory is a concern), numpy provides the keyword `out` to all of its universal functions.

This means that

In [ ]:
%memit y = np.sin(x)

In [ ]:
%memit

In [ ]:
%memit z = np.empty(len(x), dtype='float')

In [ ]:
%memit np.sin(x, out=z)

As you can see, the last input only uses half of the memory! This is because the result of `numpy.sin()` is directly stored into the already declared variable!

## Final remarks

This conludes this part of the tutorial on timing and profiling python scripts. This excercise is of course not exhaustive as python is an ongoing project still in development.

For more up to date information, hints, suggestions, please refere to the official documentation
* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)
* [Numpy user guide](https://numpy.org/doc/stable/user/index.html)